<a href="https://colab.research.google.com/github/luckyswaminathan/MusicVibe/blob/main/tweetemotionanalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import relevant modules
import numpy as np
import pandas as pd
import tensorflow as tf
import io
import string
import os
import tqdm
import time
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import nltk as nl
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
nl.download('stopwords')
nl.download('punkt')
from nltk.corpus import stopwords
nl.download('wordnet')
from wordcloud import WordCloud
import re
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from gensim.models import Word2Vec
import time
from bs4 import BeautifulSoup
#Import the DecisionTreeeClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.metrics import classification_report
#from scikitplot.metrics import plot_confusion_matrix


# #The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [69]:

#imports csv
url = 'https://raw.githubusercontent.com/luckyswaminathan/MusicVibe/main/tweet_emotions.csv'

msc_df = pd.read_csv(url)
msc_df.drop_duplicates(keep='first')
msc_df.drop(columns="tweet_id", inplace = True)



# # splitting the input/output parts



def custencoder(df):
  df.replace(to_replace="anger", value=-1, inplace=True)
  df.replace(to_replace="boredom", value=0, inplace=True)
  df.replace(to_replace="empty", value=0, inplace=True)
  df.replace(to_replace="neutral", value=0, inplace=True)
  df.replace(to_replace="enthusiasm", value=1, inplace=True)
  df.replace(to_replace="fun", value=1, inplace=True)
  df.replace(to_replace="happiness", value=1, inplace=True)
  df.replace(to_replace="hate", value=-1, inplace=True)
  df.replace(to_replace="love", value=1, inplace=True)
  df.replace(to_replace="relief", value=0, inplace=True)
  df.replace(to_replace="sadness", value=-1, inplace=True)
  df.replace(to_replace="surprise", value=1, inplace=True)
  df.replace(to_replace="worry", value=-1, inplace=True)

custencoder(msc_df['sentiment'])






rows = msc_df.shape[0]


def getTvals(tval=10000):
  neg_words = msc_df[msc_df['sentiment'] == -1].head(tval)
  pos_words = msc_df[msc_df['sentiment'] == 1].head(tval)
  
  neutral_words = msc_df[msc_df['sentiment'] == 0].head(tval)
  
  m = pd.concat([neg_words, pos_words, neutral_words])
  return m


msc_df = getTvals()

msc_df = msc_df.reindex(np.random.permutation(msc_df.index))


 #defining and removing stop words
stopWords = set(stopwords.words('english'))
stopWords.add(".")
stopWords.add(":")
stopWords.add("they're")
stopWords.add("u")
stopWords.add("im")



## msc_df['content'] = (msc_df['content']).apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopWords]))
 #tokenizing each sentence in msc_df 

def clean_text(row, options):

    if options['lowercase']:
        row = row.lower()

    if options['decode_utf8']:
        txt = BeautifulSoup(row, 'lxml')
        row = txt.get_text()

    if options['remove_url']:
        row = row.replace('http\S+|www.\S+', '')

    if options['remove_mentions']:
        row = row.replace('@[A-Za-z0-9]+', '')

    return row

clean_config = {
    'remove_url': True,
    'remove_mentions': True,
    'decode_utf8': True,
    'lowercase': True
    }

msc_df['content'] = msc_df['content'].apply(clean_text, args=(clean_config,))


tknzr = TweetTokenizer()
msc_df['content'] = msc_df['content'].apply(lambda x: re.sub('[^a-zA-Z]',' ', str(x)))
msc_df['content'] = msc_df['content'].apply(lambda x: tknzr.tokenize(x.lower()))
##msc_df['content'] = (msc_df['content']).apply(word_tokenize)

lm = WordNetLemmatizer()
msc_df['content'] = msc_df['content'].apply(lambda x: [lm.lemmatize(word) for word in x if word not in (stopWords) and word.find("quot") == -1]) 






words_train, words_test, feel_train, feel_test = train_test_split(msc_df['content'], msc_df['sentiment'], train_size=24000)


<ipython-input-69-d51e149f2547>:73: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  txt = BeautifulSoup(row, 'lxml')


In [70]:
#using word2vec to vectorize words
## using CBOW to model this data set- since the data set is 40000 rows seems to be a better fit 
## + tweets have a bunch of weird spellings so they are omitted


size = 1000
window = 5
min_count = 3
workers = 3
sg = 0 
output_folder="/usr/lakshmanswaminathan/Downloads/musicVibe/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
word2vec_model_file = os.path.join(output_folder, f'word2vec_{size}.model')
stemmed_tokens = pd.Series(words_train).values


w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, vector_size = size , workers = workers, window = window, sg = sg)
w2v_model.save(word2vec_model_file)

print(w2v_model)




  


Word2Vec<vocab=6136, vector_size=1000, alpha=0.025>


In [99]:
## working with the word2vec model/training the model


cb_w2v_model = Word2Vec.load(word2vec_model_file)

emb_df = (
    pd.DataFrame(
        [cb_w2v_model.wv.get_vector(str(n)) for n in cb_w2v_model.wv.key_to_index],
        index = cb_w2v_model.wv.key_to_index
    )
)
print(emb_df.info())
word2vec_filename = os.path.join(output_folder, 'train_word2vec.csv')

with open(word2vec_filename, 'w+') as word2vec_file:
    for index, row in msc_df.iterrows():
        model_vector = (np.mean([cb_w2v_model.wv[token] for token in row['content'] if token in cb_w2v_model.wv], axis=0)).tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(size))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(size)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

## vectors can now be trained



<class 'pandas.core.frame.DataFrame'>
Index: 6136 entries, day to jan
Columns: 1000 entries, 0 to 999
dtypes: float32(1000)
memory usage: 23.5+ MB
None


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [95]:
print(words_train.info())
print(words_test.info())
print(cb_w2v_model.predict_output_word("does this model even work"))

<class 'pandas.core.series.Series'>
Int64Index: 24000 entries, 31747 to 18099
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
24000 non-null  object
dtypes: object(1)
memory usage: 375.0+ KB
None
<class 'pandas.core.series.Series'>
Int64Index: 6000 entries, 6667 to 7649
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
6000 non-null   object
dtypes: object(1)
memory usage: 93.8+ KB
None
[('please', 0.0003281505), ('say', 0.00032809278), ('look', 0.0003265026), ('x', 0.00032536098), ('bit', 0.00032353494), ('twitter', 0.00032126915), ('guy', 0.00030563993), ('nice', 0.00030466495), ('friend', 0.00029964224), ('school', 0.0002971262)]


In [104]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Convert sentences to word embeddings using Word2Vec model



embedding_dim = cb_w2v_model.vector_size

# Define the model architecture
model = Sequential()
model.add(Dense(units=64, input_dim=1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=3, activation='softmax'))  # Assuming 3 sentiment classes

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.03),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(emb_df, msc_df['sentiment'], epochs=100, batch_size=5, validation_split=0.2)

# Evaluate the model
predictions = model.predict(feel_test)
predicted_labels = np.argmax(predictions, axis=1)
accuracy = accuracy_score(feel_test, predicted_labels)
print("Accuracy: %.2f%%" % (accuracy * 100))

Epoch 1/100


ValueError: ignored